* **앙상블 학습**은 더 좋은 예측 결과를 만들기 위해 여러 개의 모델을 훈련하는 머신러닝 알고리즘이다.
* **랜덤 포레스트**는 대표적인 결정 트리 기반의 앙상블 학습 방법이다. 부트스트랩 샘플을 사용하고 랜덤하게 일부 특성을 선택하여 트리를 만드는 것이 특징이다.
* **엑스트라 트리**는 랜덤 포레스트와 비슷하게 결정 트리를 사용하여 앙상블 모델을 만들지만 부트스트랩 샘플을 사용하지 않는다. 대신 랜덤하게 노드를 분할해 과대적합을 감소시킨다.
* **그레이디언트 부스팅**은 랜덤 포레스트나 엑스트라 트리와 달리 결정 트리를 연속적으로 추가하여 손실 함수를 최소화하는 앙상블 방법이다. 이런 이유로 훈련 속도가 조금 느리지만 더 좋은 성능을 기대할 수 있다. 그레이디언트 부스팅의 속도를 개선한 것이 **히스토그램 기반 그레이디언트 부스팅**이며 안정적인 결과와 높은 성능으로 매우 인기가 높다.

# scikit-learn

* **RandomForestClassifier**은 랜덤 포레스트 분류 클래스이다.
  * n_estimators 매개변수는 앙상블을 구성할 트리의 개수를 지정한다. 기본값은 100이다.
  * criterion 매개변수는 불순도를 지정하며 기본값은 지니 불순도를 의미하는 'gini'이고 'entropy'를 선택하여 엔트로피 불순도를 사용할 수 있다.
  * max_depth는 트리가 성장할 최대 깊이를 지정한다. 기본값은 None으로 지정하면 리프 노드가 순수하거나 min_samples_split보다 샘플 개수가 적을 때까지 성장한다.
  * min_samples_split은 노드를 나누기 위한 최소 샘플 개수이다. 기본값은 2이다.
  * max_feature 매개변수는 최적의 분할을 위해 탐색할 특성의 개수를 지정한다. 기본값은 auto로 특성 개수의 제곱근이다.
  * bootstrap 매개변수는 부트스트랩 샘플을 사용할지 지정한다. 기본값은 True이다.
  * oob_score는 OOB 샘플을 사용하여 훈련한 모델을 평가할지 지정한다. 기본값은 False이다.
  * n_jobs는 병렬 실행에 사용할 cpu 코어 수를 지정한다. 기본값은 1로 하나의 코어를 사용한다. -1로 지정하면 모든 코어를 사용한다.

* **ExtraTreesClassifier**는 엑스트라 트리 분류 클래스이다.
  * n_estimators, criterion, max_depth, min_samples_split, max_features 매개변수는 랜덤 포레스트와 동일하다.
  * bootstrap 매개변수는 부트스트랩 샘플을 사용할지 지정한다. 기본값은 False이다.
  * oob_score는 OOB 샘플을 사용하여 훈련한 모델을 평가할지 지정한다. 기본값은 False이다.

* **GradientBoostingClassifier**는 그레이디언트 부스팅 분류 클래스이다.
  * loss 매개변수는 손실 함수를 지정한다. 기본값은 로지스틱 손실 함수를 의미하는 'deviance'이다.
  * learning_rate 매개변수는 트리가 앙상블에 기여하는 정도를 조절한다. 기본값은 0.1이다.
  * n_estimators 매개변수는 부스팅 단계를 수행하는 트리의 개수이다. 기본값은 100이다.
  * subsample 매개변수는 사용할 훈련 세트의 샘플 비율을 지정한다. 기본값은 1.0이다.
  * max_depth 매개변수는 개별 회귀 트리의 최대 깊이이다. 기본값은 3이다.

* **HistGradientBoostingClassifier**는 히스토그램 기반 그레이디언트 부스팅 분류 클래스이다.
  * learning_rate 매개변수는 학습률 또는 감쇠율이다. 기본값은 0.1이며 1.0이면 감쇠가 전혀 없다.
  * max_iter는 부스팅 단계를 수행하는 트리의 개수이다. 기본값은 100이다.
  * max_bins는 입력 데이터를 나눌 구간의 개수이다. 기본값은 255이며 이보다 크게 지정할 수 없다. 여기에 1개의 구간이 누락된 값을 위해 추가된다.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)

scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [3]:
rf.fit(train_input, train_target)

print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [4]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)

print(rf.oob_score_)

0.8934000384837406


In [7]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)

socres = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [8]:
et.fit(train_input, train_target)

print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


In [9]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)

scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [10]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)

scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [11]:
gb.fit(train_input, train_target)

print(gb.feature_importances_)

[0.15887763 0.6799705  0.16115187]


In [12]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [13]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)

print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [14]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)

print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [15]:
hgb.score(test_input, test_target)

0.8723076923076923

In [16]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)

scores = cross_validate(xgb, train_input, train_target, return_train_score=True)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


In [18]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)

scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
